In [1]:
from article_lib import *

In [2]:
import glob

articles = glob.glob("..\\Articles\\1\\*.xml")

In [3]:
df = read_list_of_articles_as_dataset(articles)
df.head()

,source_id,binary_relation_id,is_revelation,title,url,body
0,1.0,1.0,0.0,"I was actually born in Kenya, says Obama",http://www.thedailymash.co.uk/news/internation...,BARACK Obama has admitted he was born in Kenya...
1,9.0,1.0,1.0,This Is How the Whole Birther Thing Actually S...,http://time.com/4496792/birther-rumor-started/,"On Friday, an old conspiracy theory about Pres..."
2,10.0,1.0,0.0,Surprise! I was born in Kenya says Barack Obama,http://rochdaleherald.co.uk/2017/01/21/surpris...,Barack Obama surprised the world today after a...
3,11.0,1.0,0.0,BARACK OBAMA'S KENYAN BIRTH CERTIFICATE,http://fourwinds10.com/siterun_data/government...,\n\this is part of what Obama has spent almost...
4,12.0,1.0,0.0,Barack Obama birthplace controversy,http://www.conservapedia.com/Barack_Obama_birt...,\n\tThe Barack Obama birthplace controversy is...


In [4]:
def join_binary_relations(df):
    path_to_binary_rels = "..\\Data\\fakes.csv"
    br = pd.read_csv(path_to_binary_rels)
    joined = pd.merge(df, br, left_on="binary_relation_id", right_on="Id")
    del joined["Id"]
    del joined["binary_relation_id"]
    del joined["Unnamed: 5"]
    return joined

In [5]:
df_joined = join_binary_relations(df)
df_joined.head()

,source_id,is_revelation,title,url,body,Subject,Predicate,Object,Link
0,1.0,0.0,"I was actually born in Kenya, says Obama",http://www.thedailymash.co.uk/news/internation...,BARACK Obama has admitted he was born in Kenya...,Barack Obama,BORN IN,Kenia,http://www.factcheck.org/2008/08/born-in-the-usa/
1,9.0,1.0,This Is How the Whole Birther Thing Actually S...,http://time.com/4496792/birther-rumor-started/,"On Friday, an old conspiracy theory about Pres...",Barack Obama,BORN IN,Kenia,http://www.factcheck.org/2008/08/born-in-the-usa/
2,10.0,0.0,Surprise! I was born in Kenya says Barack Obama,http://rochdaleherald.co.uk/2017/01/21/surpris...,Barack Obama surprised the world today after a...,Barack Obama,BORN IN,Kenia,http://www.factcheck.org/2008/08/born-in-the-usa/
3,11.0,0.0,BARACK OBAMA'S KENYAN BIRTH CERTIFICATE,http://fourwinds10.com/siterun_data/government...,\n\this is part of what Obama has spent almost...,Barack Obama,BORN IN,Kenia,http://www.factcheck.org/2008/08/born-in-the-usa/
4,12.0,0.0,Barack Obama birthplace controversy,http://www.conservapedia.com/Barack_Obama_birt...,\n\tThe Barack Obama birthplace controversy is...,Barack Obama,BORN IN,Kenia,http://www.factcheck.org/2008/08/born-in-the-usa/


In [6]:
from mitie_lib import *

import itertools
import re


def find_binary_relation_in_text(text, subject, object_, predicate):
    tokens = get_tokens(text)
    ner = get_ner()
    entities = ner.extract_entities(tokens)

    subject_positions = find_name_positions_in_text(text, entities, subject)
    object_positions = find_name_positions_in_text(text, entities, object_)
    
    if(len(subject_positions) == 0 or len(object_positions) == 0):
        return []
    
    results = []
    rel_detector = binary_relation_type_to_model(predicate)
    for subj, obj in itertools.product(subject_positions, object_positions):
        rel = ner.extract_binary_relation(tokens, subj, obj)
        
        score = rel_detector(rel)
        if (score > 0):
            results.append((subj, obj, rel))
            
    return results


def find_name_positions_in_text(text, entities, name):
    positions = []
    for entity in entities:
        entity_name = extract_text_by_xrange(text, entity[0])
        if(compare_names(name, entity_name)):
            positions.append(entity)
        
    # if our name is not Named entity, try to find it in the text
    for m in re.finditer(name, text):
        p = xrange(m.start(), m.start() + len(name))
        positions.append((p, "UNKNOWN"))

In [7]:
search_in_titles = df_joined.apply(lambda row: find_binary_relation_in_text(row['title'], row['Subject'], row['Object'], row['Predicate']), axis=1)
search_in_titles

Exception: ('Unable to load named entity extractor from D:\\Program Files\\mitieMITIE-models/english/ner_model.dat', u'occurred at index 0')

'D:\\Program Files\\mitie'